In [11]:
import os
from databricks.sdk import WorkspaceClient
from databricks.sdk.core import Config

In [30]:
local_path = "/Users/danielwilliams/Library/CloudStorage/GoogleDrive-dan@agdmalabs.com/.shortcut-targets-by-id/1EbrEbNAFF8bNuXZAUEKfw0qVdGoiHvLb/TNAU_GF_HACKATHON_2025/Paddy"

In [31]:
remote_path = "/Volumes/ue1_prod_catalog_119738067017277/tier0_sandbox/raw_tnau/Paddy"

In [32]:
original_token = os.environ.pop('DATABRICKS_TOKEN', None)
original_host = os.environ.pop('DATABRICKS_HOST', None)

config = Config(profile="agpile", http_timeout_seconds=300)
w = WorkspaceClient(config=config)

auth_headers = w.config.authenticate()

In [33]:
print(f"Scanning: {local_path}")

for root, dirs, files in os.walk(local_path):
    # Determine relative path to maintain structure
    rel_path = os.path.relpath(root, local_path)

    # Create corresponding remote directory path
    current_remote_dir = os.path.join(remote_path, rel_path)
    if rel_path == ".":
        current_remote_dir = remote_path

    # 1. Create the directory in Databricks
    try:
        w.files.create_directory(current_remote_dir)
    except Exception:
        # Ignore error if dir already exists
        pass

    for file in files:
        local_file_path = os.path.join(root, file)
        remote_file_path = os.path.join(current_remote_dir, file)
        if '.data.tif' in local_file_path:
            print(f"Skipping: {file}")
            continue
        # --- SKIP CHECK ---
        # We check if file exists to allow resuming.
        # Note: This is a simple check. For 100% rigor, we would check file size too.
        try:
            w.files.get_metadata(remote_file_path)
            print(f"[SKIP] Exists: {file}")
            continue
        except:
            # File doesn't exist, proceed to upload
            pass

        # --- UPLOAD ---
        print(f"[UPLOAD] Uploading: {remote_file_path} ...")
        try:
            with open(local_file_path, 'rb') as f:
                w.files.upload(remote_file_path, f, overwrite=True)
        except Exception as e:
            print(f"!!! FAILED: {file} - Error: {e}")

Scanning: /Users/danielwilliams/Library/CloudStorage/GoogleDrive-dan@agdmalabs.com/.shortcut-targets-by-id/1EbrEbNAFF8bNuXZAUEKfw0qVdGoiHvLb/TNAU_GF_HACKATHON_2025/Paddy
